In [1]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

#######딥러닝 라이브러리##########
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64')

In [13]:
meta_awsmap = pd.read_csv('./META/awsmap.csv')
meta_pmmap = pd.read_csv('./META/pmmap.csv')

## 사용할 aws 데이터 선별하기

In [14]:
# 가장 가까운 동네 찾기
meta_aws = meta_awsmap.loc[:,['Location','Latitude','Longitude']]
meta_pm = meta_pmmap.loc[:,['Location','Latitude','Longitude']]

In [15]:
# pm 관측별 AWS 가장 가까운 지역 선별
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # 지구의 반지름 (km)
    R = 6371.0

    # 라디안으로 변환
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # 경도, 위도 차이 계산
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine 공식 적용
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

best_aws = []
idx = ''

for index, row in meta_pm.iterrows():
    min_dist = 1e15
    location = row['Location']
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    for index, row2 in meta_aws.iterrows():
        location2 = row2['Location']
        latitude2 = row2['Latitude']
        longitude2 = row2['Longitude']
        dist = haversine(latitude, longitude, latitude2, longitude2)
        if min_dist > dist:
            min_dist = dist
            idx, loca = index, location2
        else:
            continue
            
    best_aws.append([location, idx, loca])

In [16]:
best_aws

[['아름동', 3, '세종고운'],
 ['신흥동', 5, '세종연서'],
 ['노은동', 24, '계룡'],
 ['문창동', 0, '오월드'],
 ['읍내동', 2, '장동'],
 ['정림동', 0, '오월드'],
 ['공주', 12, '공주'],
 ['논산', 14, '논산'],
 ['대천2동', 26, '대천항'],
 ['독곶리', 9, '대산'],
 ['동문동', 18, '태안'],
 ['모종동', 22, '아산'],
 ['신방동', 16, '성거'],
 ['예산군', 19, '예산'],
 ['이원면', 18, '태안'],
 ['홍성읍', 10, '홍북'],
 ['성성동', 16, '성거']]

## 데이터 불러오기

In [20]:
# Train
pm_공주 = pd.read_csv('./TRAIN/공주.csv')
pm_신방동 = pd.read_csv('./TRAIN/신방동.csv')
pm_정림동 = pd.read_csv('./TRAIN/정림동.csv')
pm_이원면 = pd.read_csv('./TRAIN/이원면.csv')
pm_읍내동 = pd.read_csv('./TRAIN/읍내동.csv')
pm_예산군 = pd.read_csv('./TRAIN/예산군.csv')
pm_아름동 = pd.read_csv('./TRAIN/아름동.csv')
pm_신흥동 = pd.read_csv('./TRAIN/신흥동.csv')
pm_성성동 = pd.read_csv('./TRAIN/성성동.csv')
pm_노은동 = pd.read_csv('./TRAIN/노은동.csv')
pm_문창동 = pd.read_csv('./TRAIN/문창동.csv')
pm_모종동 = pd.read_csv('./TRAIN/모종동.csv') 
pm_동문동 = pd.read_csv('./TRAIN/동문동.csv')
pm_독곶리 = pd.read_csv('./TRAIN/독곶리.csv')
pm_대천2동 = pd.read_csv('./TRAIN/대천2동.csv')
pm_논산 = pd.read_csv('./TRAIN/논산.csv')
pm_홍성읍 = pd.read_csv('./TRAIN/홍성읍.csv')

In [21]:
aws_공주 = pd.read_csv('./TRAIN_AWS/공주.csv')
aws_신방동 = pd.read_csv('./TRAIN_AWS/성거.csv')
aws_정림동 = pd.read_csv('./TRAIN_AWS/오월드.csv')
aws_이원면 = pd.read_csv('./TRAIN_AWS/태안.csv')
aws_읍내동 = pd.read_csv('./TRAIN_AWS/장동.csv')
aws_예산군 = pd.read_csv('./TRAIN_AWS/예산.csv')
aws_아름동 = pd.read_csv('./TRAIN_AWS/세종고운.csv')
aws_신흥동 = pd.read_csv('./TRAIN_AWS/세종연서.csv')
aws_성성동 = pd.read_csv('./TRAIN_AWS/성거.csv')
aws_노은동 = pd.read_csv('./TRAIN_AWS/계룡.csv')
aws_문창동 = pd.read_csv('./TRAIN_AWS/오월드.csv')
aws_모종동 = pd.read_csv('./TRAIN_AWS/아산.csv') 
aws_동문동 = pd.read_csv('./TRAIN_AWS/태안.csv')
aws_독곶리 = pd.read_csv('./TRAIN_AWS/대산.csv')
aws_대천2동 = pd.read_csv('./TRAIN_AWS/대천항.csv')
aws_논산 = pd.read_csv('./TRAIN_AWS/논산.csv')
aws_홍성읍 = pd.read_csv('./TRAIN_AWS/홍북.csv')

In [22]:
# Test
test_pm_공주 = pd.read_csv('./TEST_INPUT/공주.csv')
test_pm_신방동 = pd.read_csv('./TEST_INPUT/신방동.csv')
test_pm_정림동 = pd.read_csv('./TEST_INPUT/정림동.csv')
test_pm_이원면 = pd.read_csv('./TEST_INPUT/이원면.csv')
test_pm_읍내동 = pd.read_csv('./TEST_INPUT/읍내동.csv')
test_pm_예산군 = pd.read_csv('./TEST_INPUT/예산군.csv')
test_pm_아름동 = pd.read_csv('./TEST_INPUT/아름동.csv')
test_pm_신흥동 = pd.read_csv('./TEST_INPUT/신흥동.csv')
test_pm_성성동 = pd.read_csv('./TEST_INPUT/성성동.csv')
test_pm_노은동 = pd.read_csv('./TEST_INPUT/노은동.csv')
test_pm_문창동 = pd.read_csv('./TEST_INPUT/문창동.csv')
test_pm_모종동 = pd.read_csv('./TEST_INPUT/모종동.csv') 
test_pm_동문동 = pd.read_csv('./TEST_INPUT/동문동.csv')
test_pm_독곶리 = pd.read_csv('./TEST_INPUT/독곶리.csv')
test_pm_대천2동 = pd.read_csv('./TEST_INPUT/대천2동.csv')
test_pm_논산 = pd.read_csv('./TEST_INPUT/논산.csv')
test_pm_홍성읍 = pd.read_csv('./TEST_INPUT/홍성읍.csv')

In [23]:
test_aws_공주 = pd.read_csv('./TEST_AWS/공주.csv')
test_aws_신방동 = pd.read_csv('./TEST_AWS/성거.csv')
test_aws_정림동 = pd.read_csv('./TEST_AWS/오월드.csv')
test_aws_이원면 = pd.read_csv('./TEST_AWS/태안.csv')
test_aws_읍내동 = pd.read_csv('./TEST_AWS/장동.csv')
test_aws_예산군 = pd.read_csv('./TEST_AWS/예산.csv')
test_aws_아름동 = pd.read_csv('./TEST_AWS/세종고운.csv')
test_aws_신흥동 = pd.read_csv('./TEST_AWS/세종연서.csv')
test_aws_성성동 = pd.read_csv('./TEST_AWS/성거.csv')
test_aws_노은동 = pd.read_csv('./TEST_AWS/계룡.csv')
test_aws_문창동 = pd.read_csv('./TEST_AWS/오월드.csv')
test_aws_모종동 = pd.read_csv('./TEST_AWS/아산.csv') 
test_aws_동문동 = pd.read_csv('./TEST_AWS/태안.csv')
test_aws_독곶리 = pd.read_csv('./TEST_AWS/대산.csv')
test_aws_대천2동 = pd.read_csv('./TEST_AWS/대천항.csv')
test_aws_논산 = pd.read_csv('./TEST_AWS/논산.csv')
test_aws_홍성읍 = pd.read_csv('./TEST_AWS/홍북.csv')

In [6]:
submit = pd.read_csv('./answer_sample.csv')

In [24]:
# Train
cities = ['공주', '노은동', '논산', '대천2동', '독곶리',
          '동문동', '모종동', '문창동', '성성동', '신방동',
          '신흥동', '아름동', '예산군', '읍내동', '이원면',
          '정림동', '홍성읍']

for city in cities:
    globals()["pm_" + city] = globals()["pm_" + city].drop(['연도', '일시', '측정소'], axis=1)
    globals()["aws_" + city] = globals()["aws_" + city].drop(['연도','일시','지점'], axis=1)
    globals()["new_" + city] = pd.concat([globals()["aws_" + city], globals()["pm_" + city]], axis=1)
    globals()["new_" + city] = globals()["new_" + city].interpolate(method='polynomial', order=5)

    if city == '대천2동':
        globals()["new_" + city].iloc[0,-1] = globals()["new_" + city].iloc[1,-1]
    
    if city == '모종동':
        globals()["new_" + city] = globals()["new_" + city].interpolate(method='bfill')
        
for city in cities:
    globals()["test_pm_" + city] = globals()["test_pm_" + city].drop(['연도', '일시', '측정소'], axis=1)
    globals()["test_aws_" + city] = globals()["test_aws_" + city].drop(['연도','일시','지점'], axis=1).interpolate(method='polynomial', order=5)
    # globals()["test_aws_" + city] = globals()["test_aws_" + city].drop(['연도','일시','지점'], axis=1).interpolate(method='linear')
    globals()["test_new_" + city] = pd.concat([globals()["test_aws_" + city], globals()["test_pm_" + city]], axis=1)
    # globals()["test_new_" + city] = globals()["test_new_" + city].iloc[:,:-1].interpolate(method='linear')

In [29]:
n = 0
for city in cities:
    if submit[submit['측정소'] == city].isna().sum().sum() != globals()["test_new_" + city].isna().sum().sum():
        n += 1
print(n)

0


## Data Loader

In [25]:
def window_dataset(X, y, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size):
    ds_x = tf.data.Dataset.from_tensor_slices(X)
    ds_x = ds_x.window(size=X_size, stride=X_stride, shift=X_shift, drop_remainder=True)
    ds_x = ds_x.flat_map(lambda x: x.batch(X_size))
    
    ds_y = tf.data.Dataset.from_tensor_slices(y)
    ds_y = ds_y.window(size=y_size, stride=y_stride, shift=y_shift, drop_remainder=True)
    ds_y = ds_y.flat_map(lambda y: y.batch(y_size))
    
    ds = tf.data.Dataset.zip((ds_x, ds_y))
    return ds.batch(batch_size).prefetch(1)

def window_test_dataset(X, X_size, X_shift, X_stride, batch_size):
    ds_x = tf.data.Dataset.from_tensor_slices(X)
    ds_x = ds_x.window(size=X_size, stride=X_stride, shift=X_shift, drop_remainder=True)
    ds_x = ds_x.flat_map(lambda x: x.batch(X_size))
    
    return ds_x.batch(batch_size).prefetch(1)

## LSTM train & eval 

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

n_features = 5 
n_steps_in = 48
n_step_out = 72
lstm_units=16
dropout=0.2                                       

EPOCH=40
BATCH_SIZE=48  #(배치사이즈를 맞춰줌)

# X에 대한 윈도우 함수 설정값
X_size = 48
X_shift = 48
X_stride = 1

X_test_size = 48
X_test_shift = 121 # shift를 맞춰주기 위해서 48 + 72 + 1( 해당 인덱스를 포함하기 때문 )
X_test_stride = 1

# y에 대한 윈도우 함수 설정값
y_size = 72
y_shift = 72
y_stride = 1

batch_size = 48

# Train
cities = ['공주', '노은동', '논산', '대천2동', '독곶리',
          '동문동', '모종동', '문창동', '성성동', '신방동',
          '신흥동', '아름동', '예산군', '읍내동', '이원면',
          '정림동', '홍성읍']
answer = []
n = 1
for city in cities:
    print(n,'번째',city,'지역의 미세먼지 예측을 시작하겠습니다.')
    n += 1
    globals()["X_"+ city], globals()["y_"+city] = globals()["new_" + city].iloc[:,:-1].values, globals()["new_" + city].iloc[:,[-1]].values
    # globals()["X_test"+ city] = globals()["test_new_" + city][~globals()["test_new_" + city]['PM2.5'].isna()].interpolate(method='linear')
    globals()["X_test"+ city] = globals()["test_new_"+ city].iloc[:,:-1].values

    X_train, X_val, y_train, y_val = train_test_split(globals()["X_"+ city], globals()["y_"+city], test_size=0.2, random_state=42)

    train_dataset = window_dataset(X_train, y_train, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size)
    val_dataset = window_dataset(X_val, y_val, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size)

    test_dataset = window_test_dataset(globals()["X_test"+ city], X_test_size, X_test_shift, X_test_stride, batch_size)

    ######## LSTM #######
    model=Sequential([
    LSTM(lstm_units, activation='relu', input_shape=(n_steps_in, n_features)),
    # LSTM(lstm_units, activation='relu', return_sequences=True, recurrent_dropout=dropout, input_shape=(n_steps_in, n_features)),
    # LSTM(lstm_units, activation='relu'),
    Dense(n_step_out, activation='linear'),
    ])

    #######Compile 구성하기################
    model.compile(optimizer='rmsprop', loss='mae', metrics=['mae'])

    # 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
    class PrintDot(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs):
            if epoch % 10 == 0: print('')
            print('.', end='')

    #가장 좋은 성능을 낸 val_loss가 적은 model만 남겨 놓았습니다.
    save_best_only=tf.keras.callbacks.ModelCheckpoint(filepath="lstm_model.h5", monitor='val_loss', save_best_only=True)


    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

    #검증 손실이 10epoch 동안 좋아지지 않으면 학습률을 0.1 배로 재구성하는 명령어입니다.
    reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

    history = model.fit(train_dataset, epochs=EPOCH, batch_size=BATCH_SIZE, validation_data = val_dataset, verbose=1,
                    callbacks=[PrintDot(), early_stop, save_best_only , reduceLR])

    predictions = model.predict(test_dataset)
    predictions = predictions.reshape(-1,1)
    
    answer.append(predictions)

1 번째 공주 지역의 미세먼지 예측을 시작하겠습니다.
Epoch 1/40
      9/Unknown - 3s 88ms/step - loss: 0.0937 - mae: 0.0937
9/9 [==============================] - 4s 208ms/step - loss: 0.0937 - mae: 0.0937 - val_loss: 0.0788 - val_mae: 0.0788 - lr: 0.0010
Epoch 2/40
9/9 [==============================] - 1s 112ms/step - loss: 0.0732 - mae: 0.0732 - val_loss: 0.0656 - val_mae: 0.0656 - lr: 0.0010
Epoch 3/40
9/9 [==============================] - 1s 103ms/step - loss: 0.0620 - mae: 0.0620 - val_loss: 0.0579 - val_mae: 0.0579 - lr: 0.0010
Epoch 4/40
9/9 [==============================] - 1s 119ms/step - loss: 0.0557 - mae: 0.0557 - val_loss: 0.0538 - val_mae: 0.0538 - lr: 0.0010
Epoch 5/40
9/9 [==============================] - 1s 112ms/step - loss: 0.0526 - mae: 0.0526 - val_loss: 0.0519 - val_mae: 0.0519 - lr: 0.0010
Epoch 6/40
1/9 [==>...........................] - ETA: 1s - loss: 0.0515 - mae: 0.0515


KeyboardInterrupt



## LSTM with Scale & Cross validation

In [26]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

n_features = 5 
n_steps_in = 48
n_step_out = 72
lstm_units=16
dropout=0.2                                       

EPOCH= 50
BATCH_SIZE=48  #(배치사이즈를 맞춰줌)

# X에 대한 윈도우 함수 설정값
X_size = 48
X_shift = 48
X_stride = 1

# X_val_size = 48
# X_val_shift = 97 # shift를 맞춰주기 위해서 48 + 48 + 1( 해당 인덱스를 포함하기 때문 )
# X_val_stride = 1

X_test_size = 48
X_test_shift = 121 # shift를 맞춰주기 위해서 48 + 72 + 1( 해당 인덱스를 포함하기 때문 )
X_test_stride = 1

# y에 대한 윈도우 함수 설정값
y_size = 72
y_shift = 72
y_stride = 1

batch_size = 48

# Train
cities = ['공주', '노은동', '논산', '대천2동', '독곶리',
          '동문동', '모종동', '문창동', '성성동', '신방동',
          '신흥동', '아름동', '예산군', '읍내동', '이원면',
          '정림동', '홍성읍']
final_answer = []
n = 1
for city in cities:
    answer = []
    score = []
    
    print(n,'번째',city,'지역의 미세먼지 예측을 시작하겠습니다.')
    n += 1
    
    globals()["X_"+ city], globals()["y_"+city] = globals()["new_" + city].iloc[:,:-1].values, globals()["new_" + city].iloc[:,[-1]].values
    globals()["X_test"+ city] = globals()["test_new_"+ city].iloc[:,:-1].values

    scale = StandardScaler()
    globals()["X_"+ city] = scale.fit_transform(globals()["X_"+ city])
    globals()["X_test"+ city] = scale.transform(globals()["X_test"+ city])
    
    
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, val_index in tscv.split(globals()["X_"+ city]):
        X_train, X_val = globals()["X_"+ city][train_index], globals()["X_"+ city][val_index]
        y_train, y_val = globals()["y_"+city][train_index], globals()["y_"+city][val_index]

        train_dataset = window_dataset(X_train, y_train, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size)
        val_dataset = window_dataset(X_val, y_val, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size)
        test_dataset = window_test_dataset(globals()["X_test"+ city], X_test_size, X_test_shift, X_test_stride, batch_size)        
        
        # X_val_dataset = window_test_dataset(X_val, X_val_size, X_val_shift, X_val_stride, batch_size)
        # y_val_dataset = window_test_dataset(y_val, y_size, y_shift, y_stride, batch_size)

        ######## LSTM #######
        model=Sequential([
        LSTM(lstm_units, activation='relu', input_shape=(n_steps_in, n_features)),
        # LSTM(lstm_units, activation='relu', return_sequences=True, recurrent_dropout=dropout, input_shape=(n_steps_in, n_features)),
        # LSTM(lstm_units, activation='relu'),
        Dense(n_step_out, activation='linear'),
        ])

        #######Compile 구성하기################
        model.compile(optimizer='rmsprop', loss='mae', metrics=['mae'])

        # 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
        class PrintDot(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs):
                if epoch % 10 == 0: print('')
                print('.', end='')

        #가장 좋은 성능을 낸 val_loss가 적은 model만 남겨 놓았습니다.
        save_best_only=tf.keras.callbacks.ModelCheckpoint(filepath="lstm_model.h5", monitor='val_loss', save_best_only=True)

        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

        #검증 손실이 10epoch 동안 좋아지지 않으면 학습률을 0.1 배로 재구성하는 명령어입니다.
        reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

        history = model.fit(train_dataset, epochs=EPOCH, batch_size=BATCH_SIZE, validation_data = val_dataset, verbose=1,
                        callbacks=[PrintDot(), early_stop, save_best_only , reduceLR])
        
        predictions = model.predict(test_dataset)
        predictions = predictions.reshape(1,-1)
        answer.append(predictions)
        
    final_answer.append(np.mean(answer, axis = 0))

1 번째 공주 지역의 미세먼지 예측을 시작하겠습니다.
Epoch 1/50
      2/Unknown - 1s 74ms/step - loss: 0.2458 - mae: 0.2458
2/2 [==============================] - 2s 482ms/step - loss: 0.2458 - mae: 0.2458 - val_loss: 0.3533 - val_mae: 0.3533 - lr: 0.0010
Epoch 2/50
2/2 [==============================] - 0s 202ms/step - loss: 0.2354 - mae: 0.2354 - val_loss: 0.3468 - val_mae: 0.3468 - lr: 0.0010
Epoch 3/50
2/2 [==============================] - 0s 181ms/step - loss: 0.2294 - mae: 0.2294 - val_loss: 0.3420 - val_mae: 0.3420 - lr: 0.0010
Epoch 4/50
2/2 [==============================] - 0s 194ms/step - loss: 0.2248 - mae: 0.2248 - val_loss: 0.3380 - val_mae: 0.3380 - lr: 0.0010
Epoch 5/50
2/2 [==============================] - 0s 208ms/step - loss: 0.2210 - mae: 0.2210 - val_loss: 0.3346 - val_mae: 0.3346 - lr: 0.0010
Epoch 6/50
2/2 [==============================] - 0s 195ms/step - loss: 0.2177 - mae: 0.2177 - val_loss: 0.3315 - val_mae: 0.3315 - lr: 0.0010
Epoch 7/50
2/2 [==============================] - 0s

In [32]:
a = np.array(final_answer)
a = a.reshape(-1,1)
a.shape

(78336, 1)

In [31]:
a = np.array(answer)
a = a.reshape(-1,1)
a.shape

(23040, 1)

In [33]:
submit = pd.read_csv('./answer_sample.csv')
submit.iloc[:,-1] = a

In [34]:
submit.to_csv('./1_Densed_lstm_validation_epochs_80_interpolate_polynomial.csv')

## 참고자료  
시계열 데이터셋 구축  
* https://techblog-history-younghunjo1.tistory.com/373?category=922523  
  
순환신경망 레이어 구축  
* https://www.linkedin.com/pulse/multivariate-multistep-lstm-rupak-roy
* https://tykimos.github.io/2017/09/09/Time-series_Numerical_Input_Numerical_Prediction_Model_Recipe/